# Projet numérique - Calcul différentiel
Antoine Rousseau et David Castro

In [2]:
import autograd
import autograd.numpy as np

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10] # [width, height] (inches). 

from IPython.display import display

In [3]:
def grad(f):
    g = autograd.grad
    def grad_f(x, y):
        return np.array([g(f, 0)(x, y), g(f, 1)(x, y)])
    return grad_f

In [4]:
def J(f):
    j = autograd.jacobian
    def J_f(x, y):
        return np.array([j(f, 0)(x, y), j(f, 1)(x, y)]).T
    return J_f

In [5]:
def display_contour(f, x, y, levels):
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)
    fig, ax = plt.subplots()
    contour_set = plt.contour(
        X, Y, Z, colors="grey", linestyles="dashed", 
        levels=levels 
    )
    ax.clabel(contour_set)
    plt.grid(True)
    plt.xlabel("$x_1$") 
    plt.ylabel("$x_2$")
    plt.gca().set_aspect("equal")

# Question 1

La courbe de niveau $c$ de $f$ est bornée car, par hypothèse, il existe $A \in \mathbb R$ tel que :
$\forall x \in \mathbb R^2, \vert \vert x \vert \vert \geqslant A \implies f(x) \geqslant c + 1$.
Elle est donc incluse dans $\mathcal B (0, A)$.

C'est également un fermé car $\{c\}$ est un fermé de $\mathbb R$ donc, par
continuité de $f$, $f^{-1}(\{c\})$ est fermé dans $\mathbb R^2$. 

Comme on est en dimension finie, la courbe de niveau $c$ de $f$ est donc un compact de $\mathbb R^2$.

# Question 2

$ \vec u =\frac{1}{\vert \vert \nabla f(x_0)\vert \vert}(\partial_2 f(x_0), -\partial_1 f(x_0))$
est le vecteur normé tangent à la courbe de niveau en $x_0$.

# Question 3

On pose $F: (x, t) \in \mathbb R^2 \times \mathbb R \mapsto (f(x) - c, p(x) - t) \in \mathbb R^2$.

On a bien $F$ continûment dérivable car $f$ l'est et $p$ est polynômiale et $(x_0, 0)$ solution de $F(x) = 0$.

De plus, si $(x, t) \in \mathbb R^2 \times \mathbb R$,
$\partial_x F(x, t) =
\begin{pmatrix}
        \partial_1 f(x) &\partial_2 f(x) \\
        \frac{1}{\vert \vert \nabla f(x_0)\vert \vert} \partial_2 f(x_0) 
        &-\frac{1}{\vert \vert \nabla f(x_0)\vert \vert} \partial_1 f(x_0)
\end{pmatrix}$ 
de déterminant strictement négatif en $(x_0, 0)$ donc non nul sur un voisinage de $(x_0, 0)$ par continuité.

$\partial_x F$ est donc inversible sur un voisinage de $(x_0, 0)$.

Le théorème de la fonction implicite nous donne alors le résultat.

# Question 4

D'après le théorème de la fonction implicite, on a également
$\gamma'(t) = d\gamma (t) = -[\partial_x F(\gamma (t), t)]^{-1} \circ \partial_t F(\gamma (t), t)$.

Or, pour tout $(x, t) \in \mathbb R^2 \times \mathbb R$, $\partial_t F(x, t) = (0, -1)$ donc 
$\gamma' (t) = \Delta(\gamma (t)) \times (\partial_2 f(\gamma (t)), -\partial_1 f(\gamma (t)))$
où $\Delta(x) = \frac{\partial_1 f(x) \partial_1 f(x_0) + \partial_2 f(x) \partial_1 f(x_0)}
{\vert \vert \nabla f(x_0)\vert \vert}$.

Donc, par hypothèse, pour tout $t \in ]-\varepsilon, +\varepsilon[$, $\gamma'(t) \ne 0$.

On a bien $\gamma'(t)$ orthogonal à $\nabla f(\gamma(t))$.

# Question 5

In [6]:
N = 100
eps = 10**(-8)

## Tâche 1

In [7]:
def Newton(F, x0, y0, eps=eps, N=N):
    J_F = J(F)
    x, y = x0, y0
    for i in range(N):
        H = np.linalg.solve(J_F(x, y), -F(x, y))
        x += H[0]
        y += H[1]
        if np.sqrt((x - x0)**2 + (y - y0)**2) <= eps:
            return x, y
        x0, y0 = x, y
    else:
        raise ValueError(f"no convergence in {N} steps.")

## Tâche 2

In [8]:
def F(x1, x2):
    return np.array([3.0 * x1 * x1 - 2.0 * x1 * x2 + 3.0 * x2 * x2 - .8, x2 - x1])

In [9]:
Newton(F, .8, .8)

(0.4472135954999579, 0.4472135954999579)

# Question 6

## Tâche 3

In [10]:
def level_curve(f, x0, y0, delta=0.1, N=1000, eps=eps):
    xp, yp = x0, y0
    res = np.empty((2, N))
    res[:, 0] = x0, y0
    for i in range(1, N):
        def F(x, y):
            return (f(x, y) -  global c, np.sqrt((x - xp)**2 + (y - yp)**2) - delta)
        cp = Newton(F, xp, yp, eps=eps)
        res[:, i] = cp
        xp, yp = cp
    return res

# définir c
# on a pas le fait qu'il faut aller à gauche

# Question 7

## Tâche 4

In [ ]:
import matplotlib.pyplot as plt
# il faut tester avec le premier segment seulement car sinon des problèmes peuvent apparaître (courbe inachevée) comme dans le cas de la fonction de Rosenbrock.

def level_courbe_finie(f, x0, y0, delta = 0.1, N = 1000, eps = eps) :
    xp, yp = x0, y0
    res = np.empty((2, N))
    res[:, 0] = x0, y0
    for i in range(1, N):
        def F(x, y):
            return (f(x, y) - global c, np.sqrt((x - xp)**2 + (y - yp)**2) - delta)
        cp = Newton(F, xp, yp, eps=eps)
        res[:, i] = cp
        xp, yp = cp
        seg = plt.plot(res[ : i-1], res[ : i])
        for x in seg :
            if x in plt.plot(res[ : 0], res[ : 1]) :
                return res
    return res


## Tâche 5

# Question 8

Il faut :

$\begin{pmatrix} a \\ d \end{pmatrix} = \mathbf P_1$,
$\begin{pmatrix} a + b + c \\ d + e + f \end{pmatrix} = \mathbf P_2$,
$\begin{pmatrix} b \\ e \end{pmatrix} = \alpha \mathbf u_1$ et
$\begin{pmatrix} b + c \\ e + f \end{pmatrix} = \beta \mathbf u_2$
pour $a, b, c, d, e, f, g \in \mathbb R$ et $\alpha, \beta \in \mathbb R_+^*$.



Si une solution existe, $a$ et $d$ sont immédiatement déterminés.
Ce problème admet alors une solution si et seulement si les vecteurs $\mathbf P_2 - \mathbf P_1$
et $\mathbf u_2$ sont strictement postivement liés.

En effet, si ce n'est pas le cas, il est clair que le problème n'admet pas de solution.

Réciproquement, si c'est la cas, on est ramené à la résolution d'un système linéaire de 5 inconnues 
à quatre équations. Ce sytstème admet une solution, on peut même imposer $\alpha = 1$.

## Tâche 6

## Tâche 7

## Tâche 8